In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [ ]:
path = 'D:/Dataset/Competitons/Jeju/'
csv_to_parquet(path + 'train.csv', 'train')
csv_to_parquet(path + 'test.csv', 'test')

In [2]:
train = pd.read_csv(path + 'train.csv', parse_dates=["base_date"]).copy()
test = pd.read_csv(path + 'test.csv', parse_dates=["base_date"]).copy()
origin_train = train.copy()
sample_submission = pd.read_csv(path + '/sample_submission.csv').copy()
train

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,2022-06-23,목,17,0,1,106,지방도1112호선,0,0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,2022-07-28,목,21,0,2,103,일반국도11호선,0,0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,2021-10-10,일,7,0,2,103,일반국도16호선,0,0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,2022-03-11,금,13,0,2,107,태평로,0,0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,2021-10-05,화,8,0,2,103,일반국도12호선,0,0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,TRAIN_4701212,2021-11-04,목,16,0,1,107,-,0,0,...,0,대림사거리,33.422145,126.278125,없음,금덕해운,33.420955,126.273750,없음,20.0
4701213,TRAIN_4701213,2022-03-31,목,2,0,2,107,-,0,0,...,3,광삼교,33.472505,126.424368,없음,광삼교,33.472525,126.424890,없음,65.0
4701214,TRAIN_4701214,2022-06-13,월,22,0,2,103,일반국도12호선,0,0,...,0,고성교차로,33.447183,126.912579,없음,성산교차로,33.444121,126.912948,없음,30.0
4701215,TRAIN_4701215,2021-10-20,수,2,0,2,103,일반국도95호선,0,0,...,0,제6광령교,33.443596,126.431817,없음,관광대학입구,33.444996,126.433332,없음,73.0


변수명,변수 설명
1. id,아이디
2. base_date,날짜
3. day_of_week,요일
4. base_hour,시간대
5. road_in_use,도로사용여부
6. lane_count,차로수
7. road_rating,도로등급
8. multi_linked,중용구간 여부
9. connect_code,연결로 코드
10. maximum_speed_limit,최고속도제한
11. weight_restricted,통과제한하중
12. hight_restricted,통과제한높이
13. road_type,도로유형
14. start_latitude,시작지점의 위도
15. start_longitude,시작지점의 경도
16. start_turn_restricted,시작 지점의 회전제한 유무
17. end_latitude,도착지점의 위도
18. end_longitude,도착지점의 경도
19. end_turn_restricted,도작지점의 회전제한 유무
20. road_name,도로명
21. start_node_name,시작지점명
22. end_node_name,도착지점명
23. vehicle_restricted,통과제한차량
24. target,평균속도(km)

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 24 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   id                     object        
 1   base_date              datetime64[ns]
 2   day_of_week            object        
 3   base_hour              int64         
 4   road_in_use            int64         
 5   lane_count             int64         
 6   road_rating            int64         
 7   road_name              object        
 8   multi_linked           int64         
 9   connect_code           int64         
 10  maximum_speed_limit    float64       
 11  vehicle_restricted     float64       
 12  weight_restricted      float64       
 13  height_restricted      float64       
 14  road_type              int64         
 15  start_node_name        object        
 16  start_latitude         float64       
 17  start_longitude        float64       
 18  start_turn_restricted 

In [4]:
train.head(5)

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,2022-06-23,목,17,0,1,106,지방도1112호선,0,0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,2022-07-28,목,21,0,2,103,일반국도11호선,0,0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,2021-10-10,일,7,0,2,103,일반국도16호선,0,0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,2022-03-11,금,13,0,2,107,태평로,0,0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,2021-10-05,화,8,0,2,103,일반국도12호선,0,0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


In [5]:
from sklearn.preprocessing import LabelEncoder
train['day_of_week'] = LabelEncoder().fit_transform(train['day_of_week'])
train['road_name'] = LabelEncoder().fit_transform(train['road_name'])
train['start_node_name'] = LabelEncoder().fit_transform(train['start_node_name'])
train['start_turn_restricted'] = LabelEncoder().fit_transform(train['start_turn_restricted'])
train['end_node_name'] = LabelEncoder().fit_transform(train['end_node_name'])
train['end_turn_restricted'] = LabelEncoder().fit_transform(train['end_turn_restricted'])

train

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,2022-06-23,1,17,0,1,106,46,0,0,...,3,386,33.427747,126.662612,0,386,33.427749,126.662335,0,52.0
1,TRAIN_0000001,2022-07-28,1,21,0,2,103,34,0,0,...,0,32,33.500730,126.529107,1,2,33.504811,126.526240,0,30.0
2,TRAIN_0000002,2021-10-10,4,7,0,2,103,36,0,0,...,0,426,33.279145,126.368598,0,179,33.280072,126.362147,0,61.0
3,TRAIN_0000003,2022-03-11,0,13,0,2,107,57,0,0,...,0,58,33.246081,126.567204,0,203,33.245565,126.566228,0,20.0
4,TRAIN_0000004,2021-10-05,6,8,0,2,103,35,0,0,...,0,267,33.462214,126.326551,0,269,33.462677,126.330152,0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,TRAIN_4701212,2021-11-04,1,16,0,1,107,0,0,0,...,0,79,33.422145,126.278125,0,42,33.420955,126.273750,0,20.0
4701213,TRAIN_4701213,2022-03-31,1,2,0,2,107,0,0,0,...,3,31,33.472505,126.424368,0,31,33.472525,126.424890,0,65.0
4701214,TRAIN_4701214,2022-06-13,3,22,0,2,103,35,0,0,...,0,19,33.447183,126.912579,0,214,33.444121,126.912948,0,30.0
4701215,TRAIN_4701215,2021-10-20,2,2,0,2,103,37,0,0,...,0,392,33.443596,126.431817,0,23,33.444996,126.433332,0,73.0


In [6]:
test['day_of_week'] = LabelEncoder().fit_transform(test['day_of_week'])
test['road_name'] = LabelEncoder().fit_transform(test['road_name'])
test['start_node_name'] = LabelEncoder().fit_transform(test['start_node_name'])
test['start_turn_restricted'] = LabelEncoder().fit_transform(test['start_turn_restricted'])
test['end_node_name'] = LabelEncoder().fit_transform(test['end_node_name'])
test['end_turn_restricted'] = LabelEncoder().fit_transform(test['end_turn_restricted'])

test

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,...,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted
0,TEST_000000,2022-08-25,1,17,0,3,107,27,0,0,...,0.0,0,79,33.499427,126.541298,0,208,33.500772,126.543837,1
1,TEST_000001,2022-08-09,6,12,0,2,103,30,0,0,...,0.0,3,211,33.258507,126.427003,0,9,33.258119,126.415840,0
2,TEST_000002,2022-08-05,0,2,0,1,103,31,0,0,...,0.0,0,45,33.258960,126.476508,0,45,33.259206,126.474687,0
3,TEST_000003,2022-08-18,1,23,0,3,103,29,0,0,...,0.0,0,129,33.473494,126.545647,0,180,33.471061,126.545467,0
4,TEST_000004,2022-08-10,2,17,0,3,106,11,0,0,...,0.0,0,73,33.501477,126.569223,0,72,33.496863,126.581230,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291236,TEST_291236,2022-08-27,5,5,0,1,103,31,0,0,...,0.0,0,168,33.472819,126.463030,0,41,33.470483,126.460846,0
291237,TEST_291237,2022-08-19,0,20,0,2,103,29,0,0,...,0.0,0,140,33.305359,126.598914,0,230,33.300796,126.600332,0
291238,TEST_291238,2022-08-05,0,11,0,1,107,20,0,0,...,0.0,0,59,33.493624,126.496769,1,227,33.497500,126.496946,1
291239,TEST_291239,2022-08-12,0,7,0,2,107,1,0,0,...,0.0,0,16,33.254782,126.507014,0,176,33.255659,126.507333,0


In [7]:
from sklearn.ensemble import RandomForestRegressor
X_train = train.drop(['id', 'base_date', 'target'], axis = 1)
y_train = train['target'].copy()
baseline_model = RandomForestRegressor(random_state = 42, verbose = 3)
baseline_model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.9s remaining:    0.0s


building tree 2 of 100


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   44.0s remaining:    0.0s


building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 36.8min finished


RandomForestRegressor(random_state=42, verbose=3)

In [8]:
test_model = test.drop(['id', 'base_date'], axis = 1)
pred = baseline_model.predict(test_model)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    6.7s finished


In [9]:
pred

array([23.23945018, 45.13088485, 56.19831332, ..., 21.00513275,
       23.13049848, 56.96734255])

In [10]:
output = pd.DataFrame({'id' : test.id, 'target':pred})
output.to_csv('baseline.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
